# lecture 9 - Vocabulary and Matching

## rule-based matching
spaCy offers a rule-matching tool called `Matcher` that allows you to build a library of token patterns, then match those patterns against a Doc object to return a list of found matches. You can match on any part of the token including text and annotations, and you can add multiple patterns to the same matcher.

In [68]:
import spacy

In [69]:
nlp = spacy.load('en_core_web_sm')

In [70]:
from spacy.matcher import Matcher

In [71]:
matcher = Matcher(nlp.vocab)

## creating patterns

In [72]:
pattern1 = [{'LOWER':'solarpower'}]
pattern2 = [{'LOWER':'solar'},{'IS_PUNCT':True},{'LOWER':'power'}]
pattern3 = [{'LOWER': 'solar'}, {'LOWER': 'power'}]

## other token attributes
Besides lemmas, there are a variety of token attributes we can use to determine matching rules:
<table><tr><th>Attribute</th><th>Description</th></tr>

<tr ><td><span >`ORTH`</span></td><td>The exact verbatim text of a token</td></tr>
<tr ><td><span >`LOWER`</span></td><td>The lowercase form of the token text</td></tr>
<tr ><td><span >`LENGTH`</span></td><td>The length of the token text</td></tr>
<tr ><td><span >`IS_ALPHA`, `IS_ASCII`, `IS_DIGIT`</span></td><td>Token text consists of alphanumeric characters, ASCII characters, digits</td></tr>
<tr ><td><span >`IS_LOWER`, `IS_UPPER`, `IS_TITLE`</span></td><td>Token text is in lowercase, uppercase, titlecase</td></tr>
<tr ><td><span >`IS_PUNCT`, `IS_SPACE`, `IS_STOP`</span></td><td>Token is punctuation, whitespace, stop word</td></tr>
<tr ><td><span >`LIKE_NUM`, `LIKE_URL`, `LIKE_EMAIL`</span></td><td>Token text resembles a number, URL, email</td></tr>
<tr ><td><span >`POS`, `TAG`, `DEP`, `LEMMA`, `SHAPE`</span></td><td>The token's simple and extended part-of-speech tag, dependency label, lemma, shape</td></tr>
<tr ><td><span >`ENT_TYPE`</span></td><td>The token's entity label</td></tr>

</table>

In [73]:
matcher.add('SolarPower',[pattern1,pattern2,pattern3])

In [74]:
doc = nlp(u'The Solar Power industry continues to grow as demand \
for solarpower increases. Solar-power cars are gaining popularity.')

In [75]:
found_matches = matcher(doc)

In [76]:
print(found_matches)

[(8656102463236116519, 1, 3), (8656102463236116519, 10, 11), (8656102463236116519, 13, 16)]


In [77]:
for match_id, start, end in found_matches:
    string_id = nlp.vocab.strings[match_id]  # get string representation
    span = doc[start:end]                    # get the matched span
    print(match_id, string_id, start, end, span.text)

8656102463236116519 SolarPower 1 3 Solar Power
8656102463236116519 SolarPower 10 11 solarpower
8656102463236116519 SolarPower 13 16 Solar-power


In [78]:
matcher.remove('SolarPower')

## setting pattern options and quantifiers
You can make token rules optional by passing an `'OP':'*'` argument. This lets us streamline our patterns list:

In [79]:
pattern1 = [{'LOWER': 'solarpower'}] # solarpower or SolarPower
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'power'}] # solar.power or solar_power or solar,power ecc

This found both two-word patterns, with and without the hyphen!

The following quantifiers can be passed to the `'OP'` key:
<table><tr><th>OP</th><th>Description</th></tr>

<tr ><td><span >\!</span></td><td>Negate the pattern, by requiring it to match exactly 0 times</td></tr>
<tr ><td><span >?</span></td><td>Make the pattern optional, by allowing it to match 0 or 1 times</td></tr>
<tr ><td><span >\+</span></td><td>Require the pattern to match 1 or more times</td></tr>
<tr ><td><span >\*</span></td><td>Allow the pattern to match zero or more times</td></tr>
</table>


In [80]:
matcher.add('SolarPower',[pattern1,pattern2])

In [81]:
doc2 = nlp(u'Solar-power energy is solarpower!!!!')

In [82]:
found_matches = matcher(doc2)

In [84]:
print(found_matches)

[(8656102463236116519, 0, 3), (8656102463236116519, 5, 6)]


## PhraseMatcher
In the above section we used token patterns to perform rule-based matching. An alternative - and often more efficient - method is to match on terminology lists. In this case we use PhraseMatcher to create a Doc object from a list of phrases, and pass that into `matcher` instead.

In [85]:
from spacy.matcher import PhraseMatcher

In [86]:
matcher = PhraseMatcher(nlp.vocab)

In [98]:
with open('./datasets_files/reaganomics.txt') as f:
    doc3 = nlp(f.read())

In [99]:
phrase_list = ['voodoo economics', 'supply-side economics', 'trickle-down economics', 'free-market economics']

In [101]:
phrase_patterns = [nlp(text) for text in phrase_list]

In [103]:
phrase_patterns

[voodoo economics,
 supply-side economics,
 trickle-down economics,
 free-market economics]

In [106]:
matcher.add('EcoMatcher',phrase_patterns)

In [107]:
found_matches = matcher(doc3)

In [108]:
found_matches

[(2351661100535932681, 41, 45),
 (2351661100535932681, 49, 53),
 (2351661100535932681, 54, 56),
 (2351661100535932681, 61, 65),
 (2351661100535932681, 673, 677),
 (2351661100535932681, 2987, 2991)]

In [111]:
for match_id, start, end in found_matches:
    string_id = nlp.vocab.strings[match_id]  # get string representation
    span = doc3[start:end]                    # get the matched span
    print(match_id, string_id, start, end, span.text)

2351661100535932681 EcoMatcher 41 45 supply-side economics
2351661100535932681 EcoMatcher 49 53 trickle-down economics
2351661100535932681 EcoMatcher 54 56 voodoo economics
2351661100535932681 EcoMatcher 61 65 free-market economics
2351661100535932681 EcoMatcher 673 677 supply-side economics
2351661100535932681 EcoMatcher 2987 2991 trickle-down economics


In [112]:
for match_id, start, end in found_matches:
    string_id = nlp.vocab.strings[match_id]  # get string representation
    span = doc3[start-5:end+5]                    # get the matched span
    print(match_id, string_id, start, end, span.text)

2351661100535932681 EcoMatcher 41 45 policies are commonly associated with supply-side economics, referred to as trickle
2351661100535932681 EcoMatcher 49 53 economics, referred to as trickle-down economics or voodoo economics by political
2351661100535932681 EcoMatcher 54 56 trickle-down economics or voodoo economics by political opponents, and
2351661100535932681 EcoMatcher 61 65 by political opponents, and free-market economics by political advocates.


2351661100535932681 EcoMatcher 673 677 attracted a following from the supply-side economics movement, which formed in
2351661100535932681 EcoMatcher 2987 2991 became widely known as "trickle-down economics", due to the
